In [1]:
import tensorflow as tf
from l0pca.rankone import rankone_reference
from l0pca.rankone import secular_equation

2022-11-23 14:32:01.875492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
@tf.function(jit_compile=True)
def generate_data(shape):
    matrix_shape = [shape[0], 1] + list(shape[2:])
    matrix_diag = tf.random.normal(matrix_shape)
    matrix_diag += tf.math.reduce_min(matrix_diag, axis=-1)[..., None] + 0.01
    vector_update = tf.random.normal(shape)
    return matrix_diag, vector_update

In [3]:
@tf.function(jit_compile=True)
def reference_rankone_update(shape):
    eig, v = generate_data(shape)
    return rankone_reference.rankone_update_eigenvalues(eig, v)

In [7]:
%timeit reference_rankone_update((8, 512, 12))

256 ms ± 41.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
@tf.function(jit_compile=True)
def secular_impl(shape):
    eig, v = generate_data(shape)
    eig_init = secular_equation.eq_search_eigenvalues_initialize(eig, v)
    return secular_equation.eq_search_eigenvalues_iteration(eig, v, eig_init)

In [6]:
%timeit secular_impl((8, 512, 12))

2022-11-23 14:32:10.761031: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


15.7 ms ± 995 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
